### Importar Librerias

In [1]:
from binance import Client

### API KEY y Cliente

In [2]:
apikey = 'o4xp0nX8Nr3RsQIAQDBs7ZZivwpoLHPZsDQU48dmWX8heBKpSgPOS0M9NZwHHbEP'
secret = 'OwjJSwLt0szX7qq62Xqd7evsc345eCLCTrYIVTdIND4HORYJ4DDz7lmGnDGCMiwq'
client = Client(apikey, secret)

### Traer Última Orden Completada

In [ ]:
res = client.futures_get_all_orders()
status = "FILLED"

# Buscar el diccionario con el símbolo deseado
target_status = None
for position in res[::-1]:
    if position['status'] == status:
        target_status = position
        break

### Posición Actual

In [ ]:
pos_act = client.futures_position_information(symbol = target_status["symbol"])[0]

### TP Limit Según lado

In [ ]:
roe = 18

In [ ]:
aux = None
decimas = len(target_status["price"].split(".")[1])
palanca = int(pos_act["leverage"])
if target_status["side"] == "SELL":
    aux = "BUY"
    precio_salida = str(round(float(pos_act["entryPrice"])*(1-(roe/(100*palanca))),decimas))
else:
    aux = "SELL"
    precio_salida = str(round(float(pos_act["entryPrice"])*(1+(roe/(100*palanca))),decimas))

### Crear Orden TP

In [ ]:
orden_salida = client.futures_create_order(symbol = target_status["symbol"],
                            side = aux,
                            price = precio_salida,
                            quantity = abs(int(pos_act["positionAmt"])),
                            type = 'LIMIT',
                            timeinforce = 'GTC',
                            reduceOnly = True)

### Imprimir Ganancia

In [ ]:
total_salida = float(orden_salida["price"])*float(orden_salida["origQty"])
total_entrada = abs(float(pos_act["entryPrice"])*float(pos_act["positionAmt"]))
ganancia = total_salida - total_entrada
comision = total_salida*0.0002 + total_entrada*0.0002 
ganancia_neta = abs(ganancia-comision)
print("La ganancia es aproximandamente:", round(ganancia_neta,2))

### Cancelar Ordenes Abiertas

In [ ]:
client.futures_cancel_all_open_orders(symbol = target_status["symbol"])